In [410]:
import plotly.express as px
import streamlit as st
import pandas as pd 
import altair as alt
import numpy as np
from PIL import Image
import emoji


# Importing first dataframe

In [411]:
data = pd.read_csv('data.csv')
data = data[['Country', 'Continent', 'variant_grouped', 'num_sequences_total', 'date', 'Infant mortality (per 1000 births)', 'GDP ($ per capita)']]
#a = data[(data.location=='Norway') & (data.num_sequences_total.unique())]
#data[(data.location=='Norway') & (data.country==bo)]
data.head()

/var/folders/rf/s3m6tj6n4s1fnhynkdw0gbm40000gn/T/ipykernel_49613/2563494128.py:1: DtypeWarning:

Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.



,Country,Continent,variant_grouped,num_sequences_total,date,Infant mortality (per 1000 births),GDP ($ per capita)
0,Argentina,South America,Alpha,151,2020-05-11,15.18,11200.0
1,Argentina,South America,B.1,151,2020-05-11,15.18,11200.0
2,Argentina,South America,B.1.1,151,2020-05-11,15.18,11200.0
3,Argentina,South America,Beta,151,2020-05-11,15.18,11200.0
4,Argentina,South America,Delta,151,2020-05-11,15.18,11200.0


I just want the total number of cases per country

In [412]:
# Drop the variant
data = data.drop('variant_grouped', axis=1)

#data = data.groupby(['Country', 'date', 'Infant mortality (per 1000 births)', 'GDP ($ per capita)']).first()[['num_sequences_total']].sum()
#data = data.groupby(['Country',]).first().groupby(['Country'])[['num_sequences_total']].sum()

# I split the two info 
cases = data.groupby(['Country', 'Continent']).sum().num_sequences_total.reset_index().drop(["Country", "Continent"], axis =1)
info = pd.DataFrame(data.groupby(['Country',"Continent"]).first().drop(["num_sequences_total","date"],axis = 1)).reset_index()
# Concat back
data = pd.concat([info,cases], axis=1)
data.head()

,Country,Continent,Infant mortality (per 1000 births),GDP ($ per capita),num_sequences_total
0,Angola,Africa,191.19,1900.0,16880
1,Argentina,South America,15.18,11200.0,134576
2,Aruba,North America,5.89,28000.0,48000
3,Australia,Oceania,4.69,29000.0,755184
4,Austria,Europe,4.66,30000.0,201280


In [95]:
# Rounding up and renaming  
#data['num_sequences_total']=data['num_sequences_total'].apply(lambda x: round(x, 1))
#data['Total Cases per Capita']=data['Total Cases per Capita'].apply(lambda x: round(x, 7))
#data = data.rename(columns={'num_sequences_total':'Total n. of cases'})



,continent,country,tot n. of cases,Infant mortality (per 1000 births),GDP ($ per capita),Total Cases per Capita,Population,num_sequences_total
64815,North America,Jamaica,7,12.36,3900.0,0.0,2758124,1660760.1
65103,Asia,Kuwait,3,9.95,19000.0,0.0,2418393,1555118.3
65439,Asia,Vietnam,50,25.95,2500.0,0.0,84402966,9187108.7
65679,Europe,Monaco,1,5.43,27000.0,0.0,32543,180396.8
65903,Africa,Benin,57,85.00,1100.0,0.0,7862944,2804094.1


In [414]:
# Cleaning formatting
data = data.rename(columns={'num_sequences_total':'Tot number of cases'})
data.to_csv('data_gdp.csv')

# Visualizing 

In [415]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import math


In [422]:
# Importing GDP vs Infant mortality dataframe
data = pd.read_csv('data_gdp.csv', index_col=0)
data.head()

,Country,Continent,Infant mortality (per 1000 births),GDP ($ per capita),Tot number of cases
0,Angola,Africa,191.19,1900.0,16880
1,Argentina,South America,15.18,11200.0,134576
2,Aruba,North America,5.89,28000.0,48000
3,Australia,Oceania,4.69,29000.0,755184
4,Austria,Europe,4.66,30000.0,201280


In [423]:
# Plot 1

bubble_fig = px.scatter(data, x='Infant mortality (per 1000 births)', 
                                y='GDP ($ per capita)', 
                                color='Continent', 
                                title='GDP vs Infant Mortality & Covid Cases', 
                                size='Tot number of cases', 
                                log_x=True, 
                                hover_name="Country",  
                                hover_data=['GDP ($ per capita)', 'Infant mortality (per 1000 births)'],
                                size_max=70)

                                
#bubble_fig.update_layout(hovermode='closest')
bubble_fig.show()
# hovertemplaye=None
# hovermode="x unified"


In [424]:
df = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/casedistribution/csv')

In [425]:
# Remove unuseful columns
df = df[['dateRep', 'cases', 'deaths', 'countriesAndTerritories', 'countryterritoryCode', 'continentExp']]
# Rename columns
df = df.rename(columns={
    'dateRep': 'date',
    'countriesAndTerritories': 'country',
    'countryterritoryCode': 'countryCode',
    'continentExp': 'continent'
})
# Convert string to datetime
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
# Preview the data frame
df.sample(10)
df.head()

,date,cases,deaths,country,countryCode,continent
0,2020-12-14,746,6,Afghanistan,AFG,Asia
1,2020-12-13,298,9,Afghanistan,AFG,Asia
2,2020-12-12,113,11,Afghanistan,AFG,Asia
3,2020-12-11,63,10,Afghanistan,AFG,Asia
4,2020-12-10,202,16,Afghanistan,AFG,Asia


In [277]:
df.tail()

,date,cases,deaths,country,countryCode,continent
61895,2020-03-25,0,0,Zimbabwe,ZWE,Africa
61896,2020-03-24,0,1,Zimbabwe,ZWE,Africa
61897,2020-03-23,0,0,Zimbabwe,ZWE,Africa
61898,2020-03-22,1,0,Zimbabwe,ZWE,Africa
61899,2020-03-21,1,0,Zimbabwe,ZWE,Africa


In [286]:
from datetime import datetime
# Get today as string
today = datetime.now().strftime('%Y-%m-%d')
# Get a data frame only for today
df_today = df[df.date == '2020-03-25']
# Preview the data frame
df_today.head()

,date,cases,deaths,country,countryCode,continent
264,2020-03-25,2,0,Afghanistan,AFG,Asia
604,2020-03-25,23,1,Albania,ALB,Europe
885,2020-03-25,34,2,Algeria,DZA,Africa
1230,2020-03-25,31,0,Andorra,AND,Europe
1506,2020-03-25,0,0,Angola,AGO,Africa


In [338]:
# Plot 2 

fig = px.scatter_geo(
    df_today, 
    locations='countryCode',
    color='continent',
    hover_name='country',
    size='cases',
    projection="natural earth",
    title=f'World COVID-19 Cases for {today}',
)
fig.show()


# let's add the sliding bar

In [320]:
#df = pd.read_csv('https://opendata.ecdc.europa.eu/covid19/casedistribution/csv')

In [426]:
# Sort the data frame on date
df = df.sort_values(by=['date'])

In [427]:
# Convert date to string type
df['date'] = df.date.dt.strftime('%Y%m%d')
# Some countries does not have code, let's drop all the invalid rows
df = df.dropna()
# Preview the data frame
df.head(10)

,date,cases,deaths,country,countryCode,continent
28134,20191231,0,0,Iraq,IRQ,Asia
40527,20191231,0,0,Netherlands,NLD,Europe
29098,20191231,0,0,Israel,ISR,Asia
41137,20191231,0,0,New_Zealand,NZL,Oceania
16807,20191231,0,0,Dominican_Republic,DOM,America
43310,20191231,0,0,Oman,OMN,Asia
42357,20191231,0,0,North_Macedonia,MKD,Europe
42966,20191231,0,0,Norway,NOR,Europe
18657,20191231,0,0,Estonia,EST,Europe
22229,20191231,0,0,Germany,DEU,Europe


In [428]:
# Taking care of outliers
df['cases'] = df['cases'].abs()
df['deaths'] = df['deaths'].abs()
df.to_csv('cases_evolution.csv')
df.head()

,date,cases,deaths,country,countryCode,continent
28134,20191231,0,0,Iraq,IRQ,Asia
40527,20191231,0,0,Netherlands,NLD,Europe
29098,20191231,0,0,Israel,ISR,Asia
41137,20191231,0,0,New_Zealand,NZL,Oceania
16807,20191231,0,0,Dominican_Republic,DOM,America


In [392]:
import plotly.io as pio
#pio.renderers.default = 'notebook_connected'


In [407]:
fig = px.scatter_geo(
    df, 
    locations='countryCode',
    color='continent',
    hover_name='country',
    size='cases',
    projection="natural earth",
    title=f'World COVID-19 Cases - Evolution Over Time',
    animation_frame="date"
)
fig.show()